### Training of YOLO V8 Model 
### Based on https://github.com/computervisioneng/image-segmentation-yolov8


### Installed required libraries

# Link COLAB to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install Ultralytics package
!pip install ultralytics

In [ ]:
# DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/synth/Yolo'

In [ ]:
import os
from ultralytics import YOLO
import cv2, random
import numpy as np
import torch
from google.colab.patches import cv2_imshow

### Train the model 

In [ ]:
model = YOLO('yolov8n-seg.pt')  # load a pretrained model

startDirectory = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared'

script_directory = '/Training/YoloV8'

config_path = os.path.join(script_directory, 'Colab_YOLO_config.yaml')
print(config_path)
model.train(data=config_path, epochs=1)
# model.train(data=config_path, epochs=1, imgsz=640)   # Removed image size from step to see if that impacts the running 

In [ ]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

### Display a sample of images with their predicted masks 

In [ ]:

img_files = os.listdir(os.path.join(startDirectory, 'Train/Val')) # Get list of files in the directory 
for imageFile in random.sample(img_files, 3):

   results = model(imageFile)
   image = cv2.imread(imageFile)  # Replace "path_to_image.jpg" with the actual path to your image

   H, W , _ = image.shape

   # Iterate over the results
   for result in results:
      # Iterate over the masks in the current result
      for j, mask in enumerate(result.masks.data):
         #   # Move the mask tensor to CPU if it's on a CUDA device
            mask_np = mask.detach().cpu().numpy() if isinstance(mask, torch.Tensor) else mask

         #   # Convert the mask to uint8 and resize it to match the image size
            mask_np = cv2.resize((mask_np.astype(np.uint8) * 255), (image.shape[1], image.shape[0]))

         #   # Create a transparency mask
            transparency_mask = np.stack((mask_np,) * 3, axis=-1)
         
         #   # Apply the mask overlay to the original image
            overlay = cv2.addWeighted(image, 1, transparency_mask, 0.5, 0)

         #   # Display the result
            cv2_imshow(overlay)


### Copy the results from COLAB to Google Drive 

In [ ]:
import shutil

source_directory = '/content/runs'
destination_directory = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/YoloResults1'

shutil.copytree(source_directory, destination_directory)
#!scp -r /content/runs '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/YoloResults'